In [21]:
import requests
import json
import datetime
from pprint import pprint

The definition of wanikani_req is due to trunklayer. (See https://community.wanikani.com/t/python-api-call-headers-problem/45614/2)

In [132]:
def wanikani_req(token, endpoint):
    
    address = 'https://api.wanikani.com/v2'

    headers = {
        'Authorization': f'Bearer {token}'
    }

    with requests.get(f'{address}/{endpoint}', headers=headers) as r:
        response = r.json()
    #pprint(response)
    
    return response


In [174]:
def wanikani_oldies(days):
    #FUNCTION DESCRIPTION:
    #Return all assignments that were started some days ago that have NOT been burned
    
    #INITIAL VARIABLE ASSIGNMENT:
    token = '4f49ff34-8e3c-453b-9e5f-180ccf162779'
    endpoint = 'assignments?started=true'
    assignment_arr = []
    
    
    #ACQUIRING THE RESOURCES
    #First, let's get a list of all the requests. Unfortunately, we are limited to 500 resources returned for collection endpoints
    #This means if there's more than 500 resources we need to return, we need to navigate Wanikani's pagination system.
    req = wanikani_req(token, endpoint)
    requests_to_get = req['total_count']
    
    print("TOTAL REQUESTS TO GET: " + str(requests_to_get))
    
    assignment_arr = assignment_arr + req['data']
    requests_to_get = requests_to_get - len(req['data'])
    
    print("After batch 1 added...")
    print("TOTAL REQUESTS TO GET: " + str(requests_to_get))
    
    i = 1
    while requests_to_get > 0:
        i = i + 1
        
        #Now, access the info we need for the next page
        page_info = req['pages']['next_url'][40:40+24]
        
        #And modify the endpoint
        endpoint = 'assignments?' + page_info + 'started=true'
        
        #Now, make a request with the new endpoint
        req = wanikani_req(token, endpoint)
        assignment_arr = assignment_arr + req['data']
        requests_to_get = requests_to_get - len(req['data'])
        
        print("After batch " + str(i) + " added... ")
        print("TOTAL REQUESTS TO GET: " + str(requests_to_get))
    
    
    #FILTERING THE RESOURCES
    #Now, Determine of the resources we pulled, which ones correspond to assignments we started long enough ago that
    #have NOT yet been burned. These are the ones we need to study harder!
    current_datetime = datetime.datetime.now()
    cutoff_line = current_datetime - datetime.timedelta(days = days)
    
    date_str = cutoff_line.strftime("%Y-%m-%d")
    time_str = cutoff_line.strftime("%H:%M:%S")

    print("Cutoff date: " + date_str + " " + time_str)
    
    return_arr = []
    for val in assignment_arr:
        date_str = val['data']['started_at']
        date_str_1 = date_str[0:10] + ' ' + date_str[11:11+8]
        date_format = "%Y-%m-%d %H:%M:%S"
        datetime_obj = datetime.datetime.strptime(date_str_1, date_format)
        
        if str(val['data']['burned_at']) == 'None' and datetime_obj <= cutoff_line:
            print("Added resource that was started at " + date_str_1)
            return_arr.append(val)
        
        #print("New length of array: " + str(len(return_arr)))
            
    return return_arr

In [177]:
assignments = wanikani_oldies(300)

TOTAL REQUESTS TO GET: 2116
After batch 1 added...
TOTAL REQUESTS TO GET: 1616
After batch 2 added... 
TOTAL REQUESTS TO GET: 1116
After batch 3 added... 
TOTAL REQUESTS TO GET: 616
After batch 4 added... 
TOTAL REQUESTS TO GET: 116
After batch 5 added... 
TOTAL REQUESTS TO GET: 0
Cutoff date: 2022-05-03 00:29:12
Added resource that was started at 2021-10-13 04:22:19
Added resource that was started at 2021-10-10 02:53:34
Added resource that was started at 2021-10-09 23:03:56
Added resource that was started at 2021-10-27 02:02:42
Added resource that was started at 2021-11-17 04:28:29
Added resource that was started at 2021-11-24 15:18:14
Added resource that was started at 2021-11-16 07:05:12
Added resource that was started at 2021-11-16 07:05:18
Added resource that was started at 2021-11-24 15:18:29
Added resource that was started at 2021-11-28 21:39:00
Added resource that was started at 2021-11-28 21:38:57
Added resource that was started at 2021-11-28 21:38:42
Added resource that was s

43

In [102]:
len('https://api.wanikani.com/v2/assignments?')
len('page_after_id=283400700&')

24

In [ ]:
token = '4f49ff34-8e3c-453b-9e5f-180ccf162779'
endpoint = 'assignments'

In [24]:
current_datetime = datetime.datetime.now()

date_str = current_datetime.strftime("%Y-%m-%d")
time_str = current_datetime.strftime("%H:%M:%S")

print("Current date:", date_str)
print("Current time:", time_str)

Current date: 2023-02-26
Current time: 22:18:42


In [121]:
current_datetime = datetime.datetime.now()
current_datetime - datetime.timedelta(days = 180)

datetime.datetime(2022, 8, 30, 23, 23, 53, 840107)

In [126]:
date_str = '2021-10-27T02:37:38.705429Z'
date_str_1 = date_str[0:10] + ' ' + date_str[11:11+8]
date_format = "%Y-%m-%d %H:%M:%S"

datetime_obj = datetime.datetime.strptime(date_str_1, date_format)
datetime_obj

datetime.datetime(2021, 10, 27, 2, 37, 38)